In [11]:
import os
import pandas as pd
from collections import defaultdict

from IPython.display import HTML
from IPython.display import display_html

In [138]:
TPCDS_FN = "tpcds_queries_df.csv"
JOB_FN = "job_queries_df.csv"
CEB_FN = "ceb_queries_df.csv"
SCOPE_FN = "scope_queries.csv"

In [141]:
scopedf = pd.read_csv(SCOPE_FN)
scopedf["dataset"] = "SCOPE"

In [147]:
print(scopedf.keys())
scopetmp = scopedf[["dataset","like_ops", "cont_ops", "discrete_ops", "in_ops"]]

Index(['jobid', 'join', 'inputColumns', 'inputSelected', 'HashTag',
       'HashTagNonStrict', 'HashTagInput', 'HashTagRoot', 'OperatorName',
       'EstCardinality', 'InputCardinality', 'PartitioningColumn',
       'SortColumn', 'NormInput', 'NormInputHashCode', 'FilterExprNorm',
       'InclusiveTime', 'ExclusiveTime', 'VertexCount', 'RowCount',
       'Selectivity', 'QError', 'num_ops', 'num_unique_ops',
       'unique_filter_cols', 'num_cols_all', 'num_cols_sel', 'num_pcols',
       'like_ops', 'like_lens', 'like_dtype', 'like_kind', 'like_casting',
       'like_const', 'discrete_ops', 'discrete_eqs', 'discrete_noneqs',
       'discrete_type', 'discrete_types_all', 'null_checks', 'cont_ops',
       'cont_dates', 'cont_others', 'cont_type', 'cont_dtype', 'cont_range',
       'cont_cols', 'types_all', 'complex_ops', 'complex_ops_num', 'udf_ops',
       'in_ops', 'equal_dates', 'num_discrete_consts', 'dataset'],
      dtype='object')


In [148]:
tpdf = pd.read_csv(TPCDS_FN)
jobdf = pd.read_csv(JOB_FN)
cebdf = pd.read_csv(CEB_FN)
tpdf["dataset"] = "TPCDS"
jobdf["dataset"] = "JOB"
cebdf["dataset"] = "CEB"

In [149]:
print(len(tpdf))
print(len(jobdf))
print(len(cebdf))

506
706
25939


In [150]:
df = pd.concat([tpdf, jobdf, cebdf])

In [151]:
print(df.keys())

Index(['left', 'table', 'col', 'op', 'val', 'qid', 'dtype', 'dkind',
       'likedtype', 'likekind', 'discrete_ops', 'in_ops', 'cont_ops',
       'like_ops', 'dataset'],
      dtype='object')


In [152]:
# HTML(df[["like_ops", "discrete_ops", "cont_ops"]].\
#      describe(percentiles=[0.9,0.99]).reset_index().to_html(index=False))

In [153]:
def pretty_print(df):
    return display( HTML( df.to_html(index=False).replace("\\n","<br>") ) )

In [158]:
STATCOLS = ["like_ops", "discrete_ops", "cont_ops"]
STATSTODROP = ["std", "min", "max", "25%", "50%","75%"]
curdrops = []

for si, stc in enumerate(STATCOLS):
    for sdrop in STATSTODROP:
        curdrops.append(tuple([stc, sdrop]))
    if si != 0:
        curdrops.append(tuple([stc,"count"]))

#cdf = df[[""]]
cdf = df[["dataset","like_ops", "cont_ops", "discrete_ops", "in_ops"]]
cdf = pd.concat([cdf,scopetmp])

pretty_print(cdf.groupby(["dataset"])[STATCOLS].describe().\
     reset_index().\
     drop(columns=curdrops).\
     rename(columns={"like_ops":"LIKE", "discrete_ops":"Discrete", "cont_ops":"Continuous",
                    "mean":"Fraction"}).\
     round(3))

In [105]:
discdf = df[df.discrete_ops == 1]

tmp = discdf.groupby(["dataset","dkind"]).count()["discrete_ops"]
tmp = tmp.groupby(level=0).apply(lambda x: 100 * x / float(x.sum()))

# pretty_print(tmp.reset_index()\
#              .rename(columns={"discrete_ops":"Percentage"})
#             )
tmp = tmp.reset_index().set_index(['dataset', 'dkind']).rename(
    columns={"discrete_ops":"Percentage"})
tmp

Percentage
dataset dkind            
CEB     eq       7.343486
        in      92.656514
Job     eq      68.433180
        in      26.036866
        neq      5.529954
TPCDS   eq      77.922078
        in      22.077922

In [106]:
tmp = discdf.groupby(["dataset","dtype"]).count()["discrete_ops"]
tmp = tmp.groupby(level=0).apply(lambda x: 100 * x / float(x.sum()))

# pretty_print(tmp.reset_index()\
#              .rename(columns={"discrete_ops":"Percentage"})
#             )

tmp = tmp.reset_index().set_index(['dataset', 'dtype']).rename(
    columns={"discrete_ops":"Percentage"})
tmp

Percentage
dataset dtype             
CEB     date      0.129724
        string   99.870276
Job     date      5.299539
        string   94.700461
TPCDS   date     26.623377
        num      23.051948
        sql       4.545455
        string   45.779221

In [103]:
# tmp = discdf.groupby(["dataset","dtype"]).count()["discrete_ops"]
# tmp = tmp.groupby(level=0).apply(lambda x: 100 * x / float(x.sum()))
# tmp.reset_index()

In [165]:
#ldf["val"]
def get_len(val):
    val = val.replace("%", "")
    val = val.replace("'", "")
    return len(val)

In [166]:
ldf = df[df.like_ops == 1]
#ldf.groupby("likekind").count()["discrete_ops"].reset_index().sort_values(by="discrete_ops", ascending=False)

ldf["Filter Length"] = ldf.apply(lambda x: get_len(x["val"]), axis=1)
tmp = ldf.groupby(["dataset","likekind"]).count()["discrete_ops"]
tmp = tmp.groupby(level=0).apply(lambda x: 100 * x / float(x.sum()))


tmp = tmp.reset_index().set_index(['dataset', 'likekind']).rename(
    columns={"discrete_ops":"Percentage"})
tmp

<ipython-input-166-d13ad919dba7>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ldf["Filter Length"] = ldf.apply(lambda x: get_len(x["val"]), axis=1)


Percentage
dataset likekind            
CEB     contains  100.000000
JOB     3%         11.363636
        4%          0.568182
        contains   75.568182
        starts     12.500000
TPCDS   starts    100.000000

In [174]:
tmp = ldf.groupby(["dataset","likekind"])[["Filter Length"]].describe(percentiles=[0.9])
#tmp = tmp.groupby(level=0).apply(lambda x: 100 * x / float(x.sum()))
DROPS = ["std", "min", "max"]
dc = []
for dr in DROPS:
    dc.append(("Filter Length", dr))

tmp = tmp.reset_index().set_index(['dataset', 'likekind']).rename(
    columns={}).drop(columns=dc)
tmp

Filter Length                       
                         count       mean   50%   90%
dataset likekind                                     
CEB     contains        1778.0   2.347019   2.0   4.0
JOB     3%                20.0   7.700000   6.0  12.0
        4%                 1.0  11.000000  11.0  11.0
        contains         133.0   6.300752   6.0   9.8
        starts            22.0   6.727273   7.5  13.0
TPCDS   starts             1.0   5.000000   5.0   5.0

In [164]:
#ldf = df[df.like_ops == 1]

#ldf["likedtype"] = ldf.apply(lambda x: if x["likedtype"] == -1: then ,axis=1)

ldtypes = []
for idx,row in ldf.iterrows():
    if row["likedtype"] == "-1" or row["likedtype"] == -1 or row["likedtype"] == "-1.0":
        ldtypes.append("unknown")
    else:
        ldtypes.append(row["likedtype"])
ldf["likedtype"] = ldtypes

tmp = ldf.groupby(["dataset","likedtype"]).count()["discrete_ops"]
tmp = tmp.groupby(level=0).apply(lambda x: 100 * x / float(x.sum()))
tmp = tmp.reset_index().set_index(['dataset', 'likedtype']).rename(
    columns={"discrete_ops":"Percentage"})
tmp

<ipython-input-164-d00d1374220c>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ldf["Filter Length"] = ldf.apply(lambda x: get_len(x["val"]), axis=1)
<ipython-input-164-d00d1374220c>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ldf["likedtype"] = ldtypes


Percentage
dataset likedtype            
CEB     num         14.623172
        short       51.012373
        unknown     23.509561
        word        10.686164
        words        0.168729
JOB     short       10.795455
        unknown     42.045455
        word        40.909091
        words        6.250000
TPCDS   unknown    100.000000

In [109]:
#inpdata.groupby("col").count().reset_index()["val"].describe(percentiles=[0.75,0.9,0.99])

In [111]:
allinpdata = defaultdict(list)
for idx, row in df.iterrows():
    op = row["op"]
    if op in ["=", "!="]:
        allinpdata["col"].append(row["col"])
        allinpdata["val"].append(row["val"])
        allinpdata["dataset"].append(row["dataset"])

    elif op == "in":
        if "select" in row["val"]:
            pass
        else:
            cvals = row["val"]
            cvals = cvals.replace("(", "")
            cvals = cvals.replace(")", "")
            cvals = cvals.split(",")
            for cval in cvals:
                allinpdata["dataset"].append(row["dataset"])
                allinpdata["col"].append(row["col"])
                allinpdata["val"].append(cval)

In [112]:
inpdata = pd.DataFrame(allinpdata)
print(len(inpdata))
inpdata = inpdata.drop_duplicates()
inpdata = inpdata[inpdata["col"] != "function"]
print(len(inpdata))
print(len(set(inpdata["col"])))
print(set(inpdata["col"]))

68621
7768
76
{nan, 'i_size', 'hd_dep_count', 'cd_education_status', 'p_channel_dmail', 'p_channel_tv', 'd_dom', 'p_channel_event', 'year_total', 'keyword', 'd_month_seq', 'd_moy', 'd_qoy', 'dyear', 'ca_gmt_offset', 'i_brand', 's_city', 'hd_vehicle_count', 'sale_type', 'ss_store_sk', 'cc_county', 'note', 'name_pcode_nf', 'ws_item_sk', 'gender', 'kind', 'i_category', 'year', 'ca_city', 'r_reason_desc', 'name_pcode_cf', 's_county', 'name', 'i_units', 'ss_item_sk', 'ca_country', 'i_manager_id', 'i_manufact_id', 'hd_buy_potential', 's_gmt_offset', 'ws_net_profit', 'i_color', 'cs_item_sk', 'title', 'd_week_seq', 'd_quarter_name', 'cd_marital_status', 's_store_name', 'ss_net_profit', 'syear', 't_hour', 'role', 'd_dow', 'c_preferred_cust_flag', 'link', 'ca_state', 'i_item_id', 'i_item_sk', 'ca_zip', 'c_birth_country', 'd_year', 'sm_carrier', 'cd_gender', 'p_channel_email', 's_state', 'web_company_name', 'cs_qty', 'production_year', 'd_date', 'surname_pcode', 's_market_id', 'country_code', 'i_

In [113]:
inpdata.groupby("col").count().reset_index()["val"].describe(percentiles=[0.75,0.9,0.99])

count      75.000000
mean      102.813333
std       415.657853
min         1.000000
50%         3.000000
75%        11.000000
90%       139.200000
99%      1829.080000
max      3056.000000
Name: val, dtype: float64

In [129]:
STATCOLS = ["like_ops", "discrete_ops", "cont_ops"]
STATSTODROP = ["std", "min", "max", "25%", "50%","75%"]
curdrops = []

pretty_print(
    inpdata.groupby(["dataset", "col"]).nunique().reset_index()[["dataset","val"]]\
    .groupby(["dataset"])[["val"]].describe(percentiles=[0.75,0.9,0.99]).reset_index()\
    .rename(columns={"count":"Total \\n Columns", "val":"Number of unique constants"})\
    .round(3)
)